In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# データ収集用Jupyterノートブック


In [5]:
# データ収集用Jupyterノートブック

import sys
import os
import json
import datetime as dt
import pandas as pd
import asyncio

# データコレクターモジュールをインポート
from data_collector import BTCDataCollector

# 設定ファイルから読み込む
def load_config(config_path="config/data_config.json"):
    try:
        with open(config_path, "r") as f:
            config = json.load(f)

        # 日付文字列をdatetimeオブジェクトに変換
        if "start_date" in config and config["start_date"]:
            config["start_date"] = dt.datetime.fromisoformat(config["start_date"])
        if "end_date" in config and config["end_date"]:
            config["end_date"] = dt.datetime.fromisoformat(config["end_date"])
        else:
            config["end_date"] = dt.datetime.now()

        return config
    except Exception as e:
        print(f"設定ファイルの読み込みエラー: {e}")
        return None

# 設定ファイルから読み込み
config = load_config()

# または直接設定
if config is None:
    config = {
        "exchange": "bybit",
        "symbol": "BTCUSDT",
        "timeframe": "5",
        "start_date": dt.datetime(2023, 1, 1),
        "end_date": dt.datetime.now(),
        "output_dir": "data/raw",
        "output_filename": "btcusd_5m_data.csv",
        "api_keys": {
            "bybit": ["YOUR_API_KEY", "YOUR_API_SECRET"]  # 実際のキーに置き換え
        },
        "use_direct_api": True  # True: 直接APIを使用, False: pybottersを使用
    }

# API キーを設定（必要な場合）
# config["api_keys"]["bybit"] = ["YOUR_ACTUAL_API_KEY", "YOUR_ACTUAL_API_SECRET"]

# 設定内容を表示（APIキーは隠す）
print_config = config.copy()
print_config["api_keys"] = {k: ["***", "***"] for k, v in config["api_keys"].items()}
print("使用する設定:")
for key, value in print_config.items():
    print(f"  {key}: {value}")

# データ収集と保存の関数
async def collect_and_save_data(config):
    """データを収集して保存する関数"""
    collector = BTCDataCollector(config)
    df = await collector.collect_historical_data()
    summary = collector.save_data(df)

    # サマリー出力
    print("\n=== データ収集結果サマリー ===")
    print(f"状態: {summary['status']}")
    print(f"行数: {summary['rows']}")
    print(f"期間: {summary['start_date']} から {summary['end_date']}")
    if summary['status'] == 'success':
        print(f"価格統計（終値）: 最小={summary['data_stats']['close']['min']:.2f}, "
              f"最大={summary['data_stats']['close']['max']:.2f}, "
              f"平均={summary['data_stats']['close']['mean']:.2f}")
        print(f"出来高合計: {summary['data_stats']['volume']['total']:.2f}")
        print(f"欠損値: {sum(summary['missing_values'].values())}")

    return df, summary

# データ収集実行
df, summary = await collect_and_save_data(config)

# データ確認
if 'df' in locals() and not df.empty:
    print("データサンプル（先頭5行）:")
    print(df.head())

    print("\n基本統計:")
    print(df.describe())

    print(f"\n合計レコード数: {len(df)}")
else:
    print("データの収集に失敗したか、データが空です。")

2025-04-26 13:32:40,446 - bybit_collector - INFO - BTCUSDTの5分足データを2023-01-01 00:00:00から2025-04-26 13:32:40.446294まで収集します
2025-04-26 13:32:40,447 - bybit_collector - INFO - データ取得を847個のチャンクに分割しました


使用する設定:
  exchange: bybit
  symbol: BTCUSDT
  timeframe: 5
  start_date: 2023-01-01 00:00:00
  end_date: 2025-04-26 13:32:40.446294
  output_dir: data/raw
  output_filename: btcusd_5m_data.csv
  api_keys: {'bybit': ['***', '***']}
  use_direct_api: True


データ取得:   0%|          | 0/847 [00:00<?, ?it/s]

2025-04-26 13:32:55,527 - bybit_collector - INFO - データ収集完了。合計 169362 行のデータを取得
2025-04-26 13:32:56,129 - bybit_collector - INFO - データを data\raw\btcusd_5m_data.csv に保存しました。データサイズ: (169362, 6)



=== データ収集結果サマリー ===
状態: success
行数: 169362
期間: 2022-12-31T22:25:00 から 2025-04-26T04:30:00
価格統計（終値）: 最小=16508.00, 最大=109059.40, 平均=53394.16
出来高合計: 65766339.57
欠損値: 0
データサンプル（先頭5行）:
                        open     high      low    close   volume      turnover
timestamp                                                                     
2022-12-31 22:25:00  16545.0  16568.5  16544.5  16557.0  199.789  3.308472e+06
2022-12-31 22:30:00  16557.0  16569.0  16557.0  16563.0  265.757  4.402056e+06
2022-12-31 22:35:00  16563.0  16563.0  16544.5  16546.0  165.466  2.738463e+06
2022-12-31 22:40:00  16546.0  16552.0  16528.5  16551.5  295.414  4.885751e+06
2022-12-31 22:45:00  16551.5  16552.0  16533.0  16534.0   78.281  1.295245e+06

基本統計:
                open           high            low          close  \
count  169362.000000  169362.000000  169362.000000  169362.000000   
mean    53393.884619   53441.996521   53345.092630   53394.161532   
std     25365.596154   25393.485455   25337.615962  

# 特徴量エンジニアリング用Jupyter Notebook


In [6]:
# 最適化された特徴量エンジニアリングのJupyter Notebook

import sys
import os
import json
import datetime as dt
import pandas as pd
import numpy as np

# data_processorモジュールをインポート
sys.path.append(os.path.abspath(os.getcwd()))
from data_processor import generate_optimized_features

# ライブラリのインストール確認（必要に応じて）
try:
    import talib
except ImportError:
    print("TALibがインストールされていないようです。必要に応じてインストールしてください。")
    print("!pip install ta-lib  # またはta-libをソースからインストール")

# 特徴量生成の設定
config = {
    "input_dir": "data/raw",
    "input_filename": "btcusd_5m_data.csv",
    "output_dir": "data/processed",
    "output_filename": "btcusd_5m_optimized_features.csv",
    "features": {
        "price_change": True,
        "volume_change": True,
        "moving_averages": True,
        "rsi": True,
        "high_low_distance": True,
        "bollinger_bands": True,
        "macd": True,
        "stochastic": True,
        "advanced_features": True
    },
    "ma_periods": [5, 10, 20, 50, 100, 200],
    "rsi_periods": [6, 14, 24],
    "bollinger_period": 20,
    "bollinger_std": 2,
    "macd_params": {"fast": 12, "slow": 26, "signal": 9},
    "stochastic_params": {"k": 14, "d": 3, "slowing": 3},
    "atr_period": 14,
    "vwap_period": 14,
    "target_periods": [1, 2, 3],
    "classification_threshold": 0.0005
}

# 最適化された特徴量の生成を実行
print("最適化された特徴量の生成を開始します...")
feature_df, report = generate_optimized_features(config)

# 結果の表示
print("\n=== 最適化された特徴量生成結果 ===")
print(f"状態: {report['status']}")
print(f"行数: {report['rows']}")
print(f"列数: {report['columns']}")
print(f"期間: {report['start_date']} から {report['end_date']}")
print(f"欠損値: {report['missing_values']}")

print("\n特徴量グループごとの数:")
for group, count in report['feature_groups'].items():
    print(f"  {group}: {count}")

print("\n目標変数の分布:")
for target, distribution in report['target_distribution'].items():
    print(f"  {target}: {distribution}")

# 目標変数との相関が高い特徴量を表示
print("\n各期間における目標変数との重要な相関関係:")
for period in [1, 2, 3]:
    correlations = report['feature_correlations'][f'period_{period}']
    # 絶対値で相関の高い順にソート
    sorted_correlations = sorted(
        correlations.items(),
        key=lambda x: abs(x[1]),
        reverse=True
    )[:10]  # 上位10個

    print(f"\n期間 {period}:")
    for feature, corr in sorted_correlations:
        print(f"  {feature}: {corr:.6f}")

# データサンプル表示
print("\n最適化された特徴量のサンプル（主要特徴量のみ）:")
sample_columns = ['close', 'volume', 'candle_size', 'rsi_14', 'bb_width', 'stoch_k',
                 'atr', 'price_momentum', 'trend_intensity',
                 'target_price_change_pct_1', 'target_price_direction_1']
sample_columns = [col for col in sample_columns if col in feature_df.columns]
print(feature_df[sample_columns].head())

ImportError: cannot import name 'FeatureGenerator' from 'data_processor.feature_engineering' (c:\Users\trow1\Documents\my_repositories\btcusd-ml-predictor\data_processor\feature_engineering.py)

# モデルトレーニングと評価のJupyter Notebook


In [ ]:
# モデルトレーニングと評価のJupyter Notebook

import sys
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# model_builderモジュールをインポート
sys.path.append(os.path.abspath(os.getcwd()))
from model_builder import train_models, evaluate_models, format_model_report

# モデルのトレーニング設定
training_config = {
    "input_dir": "data/processed",
    "input_filename": "btcusd_5m_features.csv",
    "output_dir": "models",
    "feature_groups": {
        "price": True,            # 価格関連特徴量
        "volume": True,           # 出来高関連特徴量
        "technical": True,        # テクニカル指標関連特徴量
    },
    "target_periods": [1, 2, 3],  # 予測対象期間 (1=5分後, 2=10分後, 3=15分後)
    "cv_splits": 5,               # 時系列交差検証の分割数
    "test_size": 0.2,             # テストデータの割合
    "regression": {               # 回帰モデル（価格変動率予測）設定
        "model_params": {
            "objective": "regression",
            "metric": "mae",
            "boosting_type": "gbdt",
            "num_leaves": 31,
            "learning_rate": 0.05,
            "feature_fraction": 0.9,
            "bagging_fraction": 0.8,
            "bagging_freq": 5,
            "verbose": -1
        },
        "fit_params": {
            "num_boost_round": 1000,
            "early_stopping_rounds": 50,
            "verbose_eval": 100
        }
    },
    "classification": {           # 分類モデル（価格変動方向予測）設定
        "model_params": {
            "objective": "multiclass",
            "num_class": 3,       # 上昇/横ばい/下落の3クラス
            "metric": "multi_logloss",
            "boosting_type": "gbdt",
            "num_leaves": 31,
            "learning_rate": 0.03,  # 学習率を少し下げる
            "feature_fraction": 0.8,
            "bagging_fraction": 0.7,
            "bagging_freq": 5,
            "min_data_in_leaf": 50, # 過学習防止のために増加
            "max_depth": 8,        # 木の深さを制限
            "verbose": -1
        },
        "fit_params": {
            "num_boost_round": 2000, # ラウンド数を増加
            "early_stopping_rounds": 100,
            "verbose_eval": 100
        }
    }
}

# モデルのトレーニングを実行
print("モデルのトレーニングを開始します...")
training_report = train_models(training_config)

# トレーニング結果を表示
print("\n=== トレーニング結果 ===")

# 回帰モデルの結果
print("\n## 回帰モデル（価格変動率予測）")
for period_key, result in training_report["regression"].items():
    period = period_key.split("_")[1]
    print(f"\n### {period}期先予測")
    print(f"MAE: {result['mae']:.6f}")

    print("\n上位特徴量:")
    for feature, importance in result["top_features"].items():
        print(f"  {feature}: {importance:.6f}")

# 分類モデルの結果
print("\n## 分類モデル（価格変動方向予測）")
for period_key, result in training_report["classification"].items():
    period = period_key.split("_")[1]
    print(f"\n### {period}期先予測")
    print(f"正解率: {result['accuracy']:.4f}")

    print("\nクラスごとの精度:")
    for class_label, value in result["class_accuracy"].items():
        print(f"  {class_label}: {value:.4f}")

    print("\n上位特徴量:")
    for feature, importance in result["top_features"].items():
        print(f"  {feature}: {importance:.6f}")

# モデルの評価設定
evaluation_config = {
    "input_dir": "data/processed",
    "input_filename": "btcusd_5m_features.csv",
    "model_dir": "models",
    "output_dir": "evaluation",
    "target_periods": [1, 2, 3],  # 予測対象期間 (1=5分後, 2=10分後, 3=15分後)
    "test_size": 0.2               # テストデータの割合
}

# モデルの評価を実行
print("\nモデルの評価を開始します...")
evaluation_report = evaluate_models(evaluation_config)

# 評価結果を表示
print("\n=== 評価結果 ===")
eval_report_str = format_model_report(evaluation_report)
print(eval_report_str)